<a href="https://colab.research.google.com/github/isunitha98selvan/ner-tagger/blob/main/NER_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip3 install flair

     |████████████████████████████████| 450kB 5.6MB/s 
     |████████████████████████████████| 798kB 21.4MB/s 
     |████████████████████████████████| 1.1MB 34.5MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 983kB 40.4MB/s 
     |████████████████████████████████| 1.3MB 36.9MB/s 
     |████████████████████████████████| 19.7MB 1.5MB/s 
     |████████████████████████████████| 2.9MB 32.5MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116677 sha256=02a07f17e1d6f26cd2bd5a7e41b0b29f8b9f710512ae0df63124be6fbfe5928c
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14377 sha256=44bdcdad1575114ea369e2ca284ae451d1a2f0fe9e75f329093f0205e4f5789c
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15

In [4]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [9]:
columns = {0:'text' , 1: 'pos', 2: 'tag', 3:'ner'}
data_folder = 'gdrive/MyDrive/ner-tagger/data'

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train_data_new.txt',
                              test_file = 'test_data_new.txt',
                              dev_file = 'dev_data_new.txt')

print(len(corpus.train))
print(corpus.train[0].to_tagged_string('ner'))

2020-12-09 15:16:25,633 Reading data from gdrive/MyDrive/ner-tagger/data
2020-12-09 15:16:25,634 Train: gdrive/MyDrive/ner-tagger/data/train_data_new.txt
2020-12-09 15:16:25,635 Dev: gdrive/MyDrive/ner-tagger/data/dev_data_new.txt
2020-12-09 15:16:25,638 Test: gdrive/MyDrive/ner-tagger/data/test_data_new.txt
14392
EU <I-ORG> rejects German <I-MISC> call to boycott British <I-MISC> lamb


In [10]:
tag_type = 'ner'

tag_dict = corpus.make_tag_dictionary(tag_type=tag_type)

embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove')
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dict,
                                       tag_type=tag_type,
                                       use_crf=True)

print(tagger)

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=11, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [11]:
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50)

2020-12-09 15:16:39,532 ----------------------------------------------------------------------------------------------------
2020-12-09 15:16:39,534 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=11, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-12-09 15:16:39,536 ----------------------------------------------------------------------------------------------------
2020-12-09 15:16:39,538 Corpus: "Corpus: 14392 train + 260 dev + 3095 test sentences"
2020-12-09 15:16:39,540 ----------------------------------------------------------------------------------------------------
2020-12-09 15:16:39,543 Parameters:
2020-12-09 15:16:39,546  - learnin

{'dev_loss_history': [3.2530415058135986,
  2.5770928859710693,
  2.1141998767852783,
  2.094987154006958,
  1.8457818031311035,
  1.9226938486099243,
  1.6375597715377808,
  1.7118983268737793,
  1.6937263011932373,
  1.491698980331421,
  1.6069637537002563,
  1.5093754529953003,
  1.470401644706726,
  1.436280369758606,
  1.6286399364471436],
 'dev_score_history': [0.6959459459459459,
  0.7497360084477297,
  0.8038379530916844,
  0.8220338983050848,
  0.8508863399374348,
  0.847457627118644,
  0.8733264675592174,
  0.8414755732801595,
  0.8792569659442724,
  0.8753799392097265,
  0.874361593462717,
  0.8795180722891567,
  0.8787878787878787,
  0.8877654196157735,
  0.8902564102564102],
 'test_score': 0.8808290155440414,
 'train_loss_history': [5.28852608733707,
  3.4536277074284025,
  3.070643917189704,
  2.8424153889550103,
  2.713269590801663,
  2.5840169331762524,
  2.4726859095361498,
  2.4247847276263768,
  2.3516648128297595,
  2.2961326277256013,
  2.2415904765658907,
  2.2007